# 付録 10.1: プロンプトのチェイニング

- [レッスン](#lesson)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

「書くことは書き直すことだ」という言葉があります。実際、**Claudeはそのように求められたときに、応答の正確性を向上させることがよくあります**！

Claudeに「再考する」ように促す方法はたくさんあります。人間に自分の仕事を再確認するように自然に頼む方法は、一般的にClaudeにも効果があります。（いつ、どのように`prompt chaining`を使用するかのさらなる例については、[prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts)をご覧ください。）

### 例

この例では、Claudeに10個の単語を考え出すように頼みますが、その中の1つ以上は実際の単語ではありません。

In [ ]:
# 初期プロンプト
first_user = "全てが正確に 'ab' で終わる単語を10個挙げてください。"

# APIメッセージ配列
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Claudeの応答を保存して出力
first_response = get_completion(messages)
print(first_response)

**Claudeに回答をより正確にするように頼む**ことで、エラーが修正されます！

以下に、上記のClaudeの不正確な回答を引用し、Claudeに前の回答を修正するように頼む別のターンを追加しました。

In [ ]:
second_user = "実在しないすべての「単語」の置き換えを見つけてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

しかし、Claudeは私たちが言ったからといって答えを修正しているのでしょうか？もし最初から正しい答えを持っていたらどうなるのでしょう？Claudeは自信を失うのでしょうか？ここでは、`first_response`の場所に正しい応答を置き、再度確認するように頼みました。

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

上記のブロックから何度か応答を生成すると、Claudeはほとんどの場合、言葉をそのままにしますが、すべてが正しいにもかかわらず、時々言葉を変更することがあります。これを軽減するために何ができるでしょうか？第8章によれば、Claudeに逃げ道を与えることができます！もう一度試してみましょう。

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ フルメッセージ配列と変数の置換 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

上記のコードからの応答を何度か生成して、Claudeが自分の意見をよりしっかりと守るようになったことを確認してみてください。

また、プロンプトチェイニングを使用して、**Claudeに応答を改善させる**こともできます。以下では、まずClaudeに物語を書かせ、その後に自分が書いた物語を改善するように頼みました。個人の好みは異なるかもしれませんが、多くの人がClaudeの2回目のバージョンの方が良いと同意するかもしれません。

まず、Claudeの物語の最初のバージョンを生成してみましょう。

In [ ]:
# 初期プロンプト
first_user = "走るのが好きな女の子についての三文の短い物語を書いてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Claudeの応答を保存して出力
first_response = get_completion(messages)
print(first_response)

今、Claudeに最初のドラフトを改善させましょう。

In [ ]:
second_user = "物語を良くしてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

この形の置換は非常に強力です。私たちは、リスト、単語、Claudeの以前の応答などを渡すために置換プレースホルダーを使用してきました。また、**置換を使用して「関数呼び出し」と呼ばれることを行うことができます。これは、Claudeに何らかの関数を実行させ、その関数の結果を取得し、その結果を使ってさらにClaudeに何かをさせることです**。これは他の置換と同じように機能します。これについては次の付録で詳しく説明します。

以下は、Claudeへの1回目の呼び出しの結果を別の、より長い呼び出しに接続するもう1つの例です。まず、最初のプロンプトから始めましょう（今回はClaudeの応答を事前に入力しています）。

In [ ]:
first_user = """以下のテキストからすべての名前を見つけてください：

"こんにちは、ジェシー。私、エリンです。明日ジョーイが開くパーティーについて電話しています。キーシャは来ると言っていて、メルもそこにいると思います。"""

prefill = "<names>"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# クロードの応答を保存して出力します
first_response = get_completion(messages)
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- クロードの応答 -------------------------------------")
print(first_response)

この名前のリストを別のプロンプトに渡しましょう。

In [ ]:
second_user = "リストをアルファベット順に並べ替えます。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

プロンプトチェイニングについて学んだので、付録10.2に移動して、プロンプトチェイニングを使用した関数呼び出しの実装方法を学びましょう。

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# 初期プロンプト
first_user = "正確に「ab」で終わる単語を10個挙げてください。"

# APIメッセージ配列
messages = [
    {"role": "user",
     "content": [{"text": first_user}]}
]

# Claudeの応答を保存して出力
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "実在しないすべての「単語」の置き換えを見つけてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# 初期プロンプト
first_user = "走るのが好きな女の子についての三文の短い話を書いてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
         "content": [{"text": first_user}]
    }
]

# Claudeの応答を保存して出力
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "物語を良くしてください。"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """以下のテキストからすべての名前を見つけてください：

"こんにちは、ジェシー。私です、エリン。明日ジョーイが開くパーティーについて電話しています。キーシャは来ると言っていて、メルもそこにいると思います。"""

prefill = "<names>"

# APIメッセージ配列
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill}]
    }
]

# Claudeの応答を保存して出力する
first_response = get_completion(messages)
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(first_response)

In [ ]:
second_user = "リストをアルファベット順に並べ替えます。"

# API messages array
messages = [
    {
        "role": "user",
        "content": [{"text": first_user}]
    },
    {
        "role": "assistant",
         "content": [{"text": prefill + "\n" + first_response}]
    },
    {
        "role": "user",
         "content": [{"text": second_user}]
    },

]

# Claudeの応答を出力します
print("------------------------ 変数置換を含む完全なメッセージ配列 ------------------------")
print(messages)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(messages))